In [1]:
import torch
from torch.utils.data import Dataset, DataLoader

import numpy as np
from numpy.lib.stride_tricks import sliding_window_view
import pandas as pd
# import matplotlib.pyplot as plt
# from scipy.signal import stft, spectrogram

import os
import glob

class MyDataset(Dataset):
    def __init__(self, path):
        self.data = []
        self.label = []
        self.label1 = []
        
        paths = glob.glob(os.path.join(path, '*.csv'))
        
        for data_path in paths:     
               
            # if not ('backward' in data_path or 'forward' in data_path):
            #     continue
                
            print(data_path)
          
            # 读取CSV数据
            data = pd.read_csv(data_path, header=None)
            data = data.iloc[::4, :]

            # time_stamps = data.iloc[:, 0].values  # 时间戳列
            data = data.iloc[:, 1:]
            # data = data.iloc[:, :3]

            data = sliding_window_view(data, 60, axis=0)
            
            self.data.append(data)

            self.label.extend([0 if 'standing' in data_path else \
                               0 if 'closed' in data_path else \
                               1 if 'oneleg' in data_path else \
                               2 ] * data.shape[0])
            
            self.label1.extend([0 if 'forward' in data_path else \
                                1 if 'backward' in data_path else 2] * data.shape[0])         
        
        self.data = np.concatenate(self.data)
        self.label = np.array(self.label)
        
        self.label1 = np.array(self.label1)
        
        print(self.data.shape)
        print(self.label.shape)
        
        print()
        
        for i in range(3):
            print(np.sum(self.label == i))
        
        self.data = torch.tensor(self.data, dtype=torch.float32)
        self.label = torch.tensor(self.label, dtype=torch.long)
        self.label1 = torch.tensor(self.label1, dtype=torch.long)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx], self.label[idx], self.label1[idx]
    
data = MyDataset('collect_data/data_new')

split_ratio = 0.8
train_data, valid_data = torch.utils.data.random_split(data, [int(len(data)*split_ratio), len(data)-int(len(data)*split_ratio)])

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=32, shuffle=False)

print(len(train_data), len(valid_data))

collect_data/data_new\czl_backward.csv
collect_data/data_new\czl_closed.csv
collect_data/data_new\czl_forward.csv
collect_data/data_new\czl_oneleg1.csv
collect_data/data_new\czl_oneleg2.csv
collect_data/data_new\czl_standing.csv
collect_data/data_new\leon_backward.csv
collect_data/data_new\leon_closed.csv
collect_data/data_new\leon_forward.csv
collect_data/data_new\leon_oneleg.csv
collect_data/data_new\leon_oneleg2.csv
collect_data/data_new\leon_standing.csv
(17063, 6, 60)
(17063,)

5673
5638
5752
13650 3413


In [2]:
valid_data[0][0].shape

torch.Size([6, 60])

In [3]:
valid_data[3][0]

tensor([[ 4.1748e-02, -3.7354e-02,  6.5674e-02,  1.0474e-01,  1.6040e-01,
          1.7358e-01,  1.3892e-01,  1.2549e-01,  8.4961e-02,  4.4434e-02,
          3.4180e-02,  4.9316e-02,  5.6396e-02,  9.9365e-02,  1.4917e-01,
          9.7656e-02,  6.6162e-02,  8.9111e-02,  8.1787e-02,  1.3379e-01,
          7.4707e-02,  9.9854e-02,  2.0801e-01,  1.4526e-01,  1.6602e-02,
          7.6904e-02,  6.9092e-02,  2.3926e-02, -5.1270e-03,  3.9307e-02,
          1.6016e-01,  2.0825e-01,  1.0840e-01,  1.2134e-01,  1.5039e-01,
         -4.8800e-04,  5.2490e-02,  2.0679e-01,  1.9458e-01,  1.5527e-01,
          1.0596e-01,  1.5991e-01,  2.5244e-01,  1.6162e-01,  2.7124e-01,
          2.6172e-01,  1.6260e-01,  1.3330e-01,  1.8408e-01,  2.3975e-01,
          2.4536e-01,  3.0395e-01,  2.9370e-01,  3.0347e-01,  3.3862e-01,
          2.9224e-01,  2.7051e-01,  1.6724e-01,  2.8613e-01,  2.3291e-01],
        [-3.0066e-01, -3.4119e-01, -2.7576e-01, -2.3743e-01, -2.4719e-01,
         -3.3533e-01, -3.1799e-01, -3

In [4]:
import torch
import torch.nn as nn


from models import BWClassifier

In [5]:
from torchinfo import summary
model = BWClassifier()
summary(model, (1, 6, 60))

Layer (type:depth-idx)                   Output Shape              Param #
BWClassifier                             [1, 4]                    --
├─Conv1d: 1-1                            [1, 4, 30]                124
├─ReLU: 1-2                              [1, 4, 30]                --
├─Conv1d: 1-3                            [1, 6, 30]                78
├─ReLU: 1-4                              [1, 6, 30]                --
├─Flatten: 1-5                           [1, 180]                  --
├─Linear: 1-6                            [1, 4]                    724
├─Linear: 1-7                            [1, 2]                    362
Total params: 1,288
Trainable params: 1,288
Non-trainable params: 0
Total mult-adds (M): 0.01
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.01
Estimated Total Size (MB): 0.01

In [6]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
model = BWClassifier().to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4)
loss_func = nn.CrossEntropyLoss()

In [7]:
# n_epochs = 20

# for epoch in range(n_epochs):
#     model.train()
#     for batch_idx, (data, target) in enumerate(train_loader):
#         data, target = data.to(device), target.to(device)
        
#         optimizer.zero_grad()
#         output = model(data)
        
#         loss = loss_func(output, target)
        
#         loss.backward()
#         optimizer.step()
        
#         # 打印训练信息
#         if batch_idx % 100 == 0:
#             print(f'Epoch [{epoch+1}/{n_epochs}], Step [{batch_idx}/{len(train_loader)}], Loss: {loss.item():.4f}')
            
#     model.eval()
    
#     valid_acc = 0.0
#     valid_size = 0
    
#     with torch.inference_mode():
#         for batch_idx, (data, target) in enumerate(valid_loader):
#             data, target = data.to(device), target.to(device)
#             output = model(data)
        
#             loss = loss_func(output, target)
            
#             # target = target.argmax(dim=1)
#             pred = output.argmax(dim=1)
#             valid_acc += pred.eq(target).sum().item()
#             valid_size += target.size(0)
            
#             # 打印验证信息
#             if batch_idx % 100 == 0:
#                 print(f'Validation, Step [{batch_idx}/{len(valid_loader)}], Loss: {loss.item():.4f}')
                
#     # valid_acc = (100. * valid_acc) / valid_size
#     valid_acc = (100. * valid_acc) / valid_size
    
#     print(f'Validation Accuracy: {valid_acc:.4f}')

n_epochs = 30

for epoch in range(n_epochs):
    model.train()
    for batch_idx, (data, target1, target2) in enumerate(train_loader):
        data, target1, target2 = data.to(device), target1.to(device), target2.to(device)
        
        optimizer.zero_grad()
        output1, output2 = model(data)
        
        loss = loss_func(output1, target1)
        mask = target2 != 2
        output2 = output2[mask]
        target2 = target2[mask]
        loss += loss_func(output2, target2)
        
        loss.backward()
        optimizer.step()
        
        # 打印训练信息
        if batch_idx % 100 == 0:
            print(f'Epoch [{epoch+1}/{n_epochs}], Step [{batch_idx}/{len(train_loader)}], Loss: {loss.item():.4f}')
            
    model.eval()
    
    valid_acc1 = 0.0
    valid_size1 = 0
    
    valid_acc2 = 0.0
    valid_size2 = 0
    with torch.inference_mode():
        for batch_idx, (data, target1, target2) in enumerate(valid_loader):
            data, target1, target2 = data.to(device), target1.to(device), target2.to(device)
            output1, output2 = model(data)
        
            loss = loss_func(output1, target1)
            mask = target2 != 2
            output2 = output2[mask]
            target2 = target2[mask]
            loss += loss_func(output2, target2)
            
            # target = target.argmax(dim=1)
            pred1 = output1.argmax(dim=1)
            valid_acc1 += pred1.eq(target1).sum().item()
            valid_size1 += target1.size(0)
            
            pred2 = output2.argmax(dim=1)
            valid_acc2 += pred2.eq(target2).sum().item()
            valid_size2 += target2.size(0)
            
            # 打印验证信息
            if batch_idx % 100 == 0:
                print(f'Validation, Step [{batch_idx}/{len(valid_loader)}], Loss: {loss.item():.4f}')
                
    # valid_acc = (100. * valid_acc) / valid_size
    valid_acc1 = (100. * valid_acc1) / valid_size1
    valid_acc2 = (100. * valid_acc2) / valid_size2
    
    print(f'Validation Accuracy: {valid_acc1:.4f}, {valid_acc2:.4f}')

Epoch [1/30], Step [0/427], Loss: 2.8453
Epoch [1/30], Step [100/427], Loss: 2.3101
Epoch [1/30], Step [200/427], Loss: 1.7140
Epoch [1/30], Step [300/427], Loss: 1.6065
Epoch [1/30], Step [400/427], Loss: 1.3827
Validation, Step [0/107], Loss: 1.3963
Validation, Step [100/107], Loss: 1.4895
Validation Accuracy: 84.2074, 47.9203
Epoch [2/30], Step [0/427], Loss: 1.2904
Epoch [2/30], Step [100/427], Loss: 1.3434
Epoch [2/30], Step [200/427], Loss: 1.2670
Epoch [2/30], Step [300/427], Loss: 1.0691
Epoch [2/30], Step [400/427], Loss: 0.9630
Validation, Step [0/107], Loss: 0.9265
Validation, Step [100/107], Loss: 1.0087
Validation Accuracy: 89.2763, 74.7834
Epoch [3/30], Step [0/427], Loss: 0.9901
Epoch [3/30], Step [100/427], Loss: 0.8710
Epoch [3/30], Step [200/427], Loss: 0.9346
Epoch [3/30], Step [300/427], Loss: 0.9248
Epoch [3/30], Step [400/427], Loss: 0.7671
Validation, Step [0/107], Loss: 0.5264
Validation, Step [100/107], Loss: 0.6744
Validation Accuracy: 89.9209, 86.1352
Epoch [

In [8]:
torch.save(model.state_dict(), 'model.pth')

In [9]:
data = MyDataset('collect_data/data')
model = model.to('cpu')

collect_data/data\leon_backward.csv
collect_data/data\leon_closed.csv
collect_data/data\leon_forward.csv
collect_data/data\leon_oneleg.csv
collect_data/data\leon_oneleg1.csv
collect_data/data\leon_oneleg2.csv
collect_data/data\leon_shaking.csv
collect_data/data\leon_standing.csv
collect_data/data\li_backward.csv
collect_data/data\li_closed.csv
collect_data/data\li_forward.csv
collect_data/data\li_oneleg.csv
collect_data/data\li_shaking.csv
collect_data/data\li_standing.csv
collect_data/data\mark_oneleg1.csv
collect_data/data\mark_oneleg2.csv
(11765, 6, 60)
(11765,)

2873
4187
4705


In [10]:
wrong = 0
for i in range(len(data)):
    x, y1, y2 = data[i]
    pred1, pred2 = model(x.unsqueeze(0))
    
    if pred2.argmax().item() >= 2:
        print('??')
        
    if (y2 != 2 and pred2.argmax().item() != y2) or pred1.argmax().item() != y1:
        print(pred1.argmax().item(), y1.item())
        print(pred2.argmax().item(), y2.item())
        print()
        wrong += 1         
    
print(len(data), wrong)

# x, y1, y2 = data[532]
# pred1, pred2 = model(x.unsqueeze(0))

# print(pred1.argmax(), y1)
# print(pred2.argmax(), y2)

2 2
0 1

2 2
0 1

2 2
0 1

2 2
0 1

2 2
0 1

2 2
0 1

2 2
0 1

2 2
0 1

2 2
0 1

2 2
0 1

2 2
0 1

2 2
0 1

2 2
0 1

2 2
0 1

2 2
0 1

1 2
1 1

1 2
1 1

1 2
1 1

1 2
1 1

1 2
1 1

1 2
1 1

1 2
1 1

1 2
1 1

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
0 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
0 2

2 1
0 2

2 1
0 2

2 1
1 2

2 1
0 2

2 1
1 2

2 1
0 2

2 1
0 2

2 1
0 2

2 1
0 2

2 1
0 2

2 1
0 2

2 1
0 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2 1
1 2

2

In [11]:
import torch
import numpy as np

# 將 tensor 轉換成 numpy array

d_in = data[10][0]

from models import BWClassifier

model = BWClassifier()
model.load_state_dict(torch.load('model.pth'))
model.eval()

print(model(d_in.unsqueeze(0)))

array = d_in.numpy()

# 將 numpy array 寫入 C header file
with open('tensor_data.h', 'w') as f:
    f.write('#ifndef TENSOR_DATA_H\n')
    f.write('#define TENSOR_DATA_H\n\n')
    
    f.write('const float tensor_data[{}][{}] = {{\n'.format(array.shape[0], array.shape[1]))
    for row in array:
        f.write('    {')
        f.write(', '.join(map(str, row)))
        f.write('},\n')
    f.write('};\n\n')
    
    f.write('#endif // TENSOR_DATA_H\n')

(tensor([[-12.3451,  -2.3217,   6.4715, -39.7537]], grad_fn=<AddmmBackward0>), tensor([[-8.0880,  9.4643]], grad_fn=<AddmmBackward0>))
